Project 22

In [2]:
# data load


In [3]:
# simple CNN model

# import libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class PrintSize(nn.Module):
    """Utility module to print current shape of a Tensor in Sequential, only at the first pass."""
    
    first = True
    
    def forward(self, x):
        if self.first:
            print(f"Size: {x.size()}")
            self.first = False
        return x

# create model as a class using pytorch
# 11 channel input, one channel output
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #5x5 square convolution
        self.conv1 = nn.Conv2d(in_channels=11, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=1)

        #reLU activation
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()

        #max pooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        
        #convolution 1
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        #convolution 2
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        #convolution 3
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        #flatten
        x = x.view(-1, 16 * 5 * 5)

        #fully connected layers
        x = self.fc1(x)
        return x

        # # Max pooling over a (2, 2) window
        # x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # # If the size is a square you can only specify a single number
        # x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # x = x.view(-1, self.num_flat_features(x))
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        # return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)


Net(
  (conv1): Conv2d(11, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (relu2): ReLU()
  (relu3): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)


### **Loss Function and Optimizer**


In [4]:
loss_fn = nn.CrossEntropyLoss()
# optimizer
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


### **Test network before training**

In [5]:
out = model(torch.randn(1, 11, 100, 100, device=device))

print(f"Output logits:\n{out.detach().cpu().numpy()}")
print(f"Output probabilities:\n{out.softmax(1).detach().cpu().numpy()}")

Output logits:
[[ 0.03886781 -0.02025883 -0.00691133 ...  0.04848625  0.00193986
  -0.00317327]
 [-0.16059351 -0.06139738 -0.00373689 ...  0.00537689  0.02744661
   0.12507975]
 [-0.02228721 -0.04071133  0.08149298 ... -0.02612013 -0.07958535
   0.07468948]
 ...
 [-0.25064653 -0.06748635  0.00727729 ... -0.04061291  0.044673
   0.0860717 ]
 [-0.12622061 -0.11766759  0.08000813 ... -0.03136643 -0.03241026
   0.11948291]
 [ 0.02717645 -0.09066263  0.01199031 ...  0.01981709 -0.05321397
  -0.00031   ]]
Output probabilities:
[[0.0086699  0.00817213 0.00828194 ... 0.00875369 0.00835557 0.00831296]
 [0.00700331 0.00773363 0.00819267 ... 0.00826767 0.00845217 0.00931901]
 [0.00809448 0.00794671 0.00897967 ... 0.00806352 0.00764372 0.00891878]
 ...
 [0.00640943 0.00769777 0.00829534 ... 0.00790744 0.00861142 0.00897541]
 [0.00726263 0.00732501 0.00892602 ... 0.00798525 0.00797692 0.00928542]
 [0.0085669  0.0076146  0.00843779 ... 0.00850409 0.00790516 0.00833463]]


### **Training the network**

In [ ]:
batch_size = 64
num_epochs = 10
validation_every_n = 100

step = 0
model.train()




Net(
  (conv1): Conv2d(11, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (relu2): ReLU()
  (relu3): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)